In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from pathlib import Path
import pandas as pd
# import researchpy as rp
import statsmodels.api as sm
import scipy.stats as stats
import statsmodels.formula.api as smf
import numpy as np
import statsmodels.api as sm
import os
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

from decimal import Decimal
import statsmodels.formula.api
from sklearn import metrics

import anndata as ad
import pandas as pd
from sklearn.cluster import KMeans
# import scanpy as sc
import anndata as ad
import numpy as np
import plotly.express as px
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import scipy.stats
from scipy import stats
import numpy as np
from sklearn.decomposition import NMF, PCA
from sklearn.decomposition import FastICA
import seaborn as sns; sns.set_theme(color_codes=True)
# import umap
from sklearn.mixture import GaussianMixture
from numpy.random import randn
from numpy.random import seed
from scipy.stats import pearsonr

from tqdm import tqdm
import statsmodels
# from PyPDF2 import PdfFileMerger


In [2]:
sns.set(context='notebook', style='ticks', font_scale=2, font='sans-serif', rc={"lines.linewidth": 1.2})
sns.set_style("white")

import numpy as np
import scipy.stats as st
def ci_95(data):
    x = st.t.interval(alpha=0.95, df=len(data)-1, loc=np.mean(data), scale=st.sem(data)) 
    return f"{((x[0]+x[1]) / 2).round(2)} [{x[0].round(2)}-{x[1].round(2)}][{len(data)}]"

In [3]:
phenotypes = pd.read_csv("/Users/projects_data/project_MLPhenotypesMRIGWAS/organized_data/23_03_30/Ensemble_MLphenotypes_cleaned.csv")
phenotypes = phenotypes.set_index('index')

In [4]:
!pwd

/Users/dadua2/EssentialCodeBase/project_MLPhenotypesMRIGWAS/figures4paper/0.FINALFOLDER


In [7]:
from pathlib import Path
import os
import anndata as ad

gen_dir = Path('/Users/projects_data/project_MLPhenotypesMRIGWAS/ml_generated_data/23_03_30')
adata = ad.read(gen_dir / "preprocessed_data" / "adata.h5ad" )
traw_data1 = pd.DataFrame(adata.X, index=adata.obs.index, columns=adata.var.index)
traw_data2 = adata.obs
raw_data = pd.merge(traw_data1, traw_data2, left_index=True, right_index=True)
invicro_features_filtered = [] # sorted(list(adata.var.index))
raw_data = raw_data.dropna(subset=invicro_features_filtered+['age',  'patid'])
raw_data['study_diagnosis'] = raw_data['study'].astype(str) + '_' + raw_data['diagnosis'].astype(str)
raw_data['subject_id'] = raw_data['patid'].values
raw_data['sid'] = raw_data.index

J = pd.read_csv(Path('/Users/EssentialCodeBase/data/clinical/raw_data/23_03_unknown') / "ADNI_cov.tsv", sep='\t').copy()
J['USERDATE'] = pd.to_datetime(J['USERDATE'])
req_columns = ['PTDOBYY', 'PTCOGBEG', 'PTMCIBEG', 'PTADBEG', 'PTADDX']
for col in req_columns:
    temp = J[(J[col] > 2100) | (J[col] < 1900)].index
    J.loc[temp, col] = np.nan
    
JX_adni = J.groupby('participant_id').agg('min')
JX_adni['age_COGBEG'] = JX_adni['PTCOGBEG'] - JX_adni['PTDOBYY']
JX_adni['age_MCIBEG'] = JX_adni['PTMCIBEG'] - JX_adni['PTDOBYY']
JX_adni['age_ADBEG'] = JX_adni['PTADBEG'] - JX_adni['PTDOBYY']
JX_adni['age_ADDX'] = JX_adni['PTADDX'] - JX_adni['PTDOBYY']
JX_adni['age_MCI|COGBEG'] = JX_adni[['age_COGBEG', 'age_MCIBEG']].agg('min', axis=1)
JX_adni = JX_adni[['age_COGBEG', 'age_MCIBEG', 'age_ADBEG', 'age_ADDX', 'age_MCI|COGBEG', 'age_at_baseline', 'PTDOBYY']].copy()

JX_adni.index = pd.Series(JX_adni.index.map(lambda x: f'ADNI-{x}'), name='subject_id')
JX_adni = JX_adni.reset_index()
T = pd.read_csv("/Users/EssentialCodeBase/data/clinical/raw_data/23_03_unknown/Screening___Demographics.csv")
birth_year = dict(zip(T['PATNO'].values, T['BIRTHDT'].values))
J = pd.read_csv(Path('/Users/EssentialCodeBase/data/clinical/raw_data/23_03_unknown') / "processed_uniq_visit_month_mean_U3.csv")
J = J[['participant_id', 'age_at_baseline', 'age_at_onset', 'age_at_diagnosis']]
JX_ppmi = J.groupby('participant_id').agg('min')
JX_ppmi['PTDOBYY'] = JX_ppmi.index.map(birth_year)
JX_ppmi.index = pd.Series(JX_ppmi.index.map(lambda x: f'PPMI-{x}'), name='subject_id')
JX_ppmi = JX_ppmi.reset_index()
JX = pd.concat([JX_adni, JX_ppmi], axis=0).copy()

In [8]:
adni_ppmi = pd.merge(raw_data[~(raw_data.index.str.contains('UKBB'))], JX, left_on='subject_id', right_on='subject_id', how='left')
adni_ppmi = adni_ppmi.dropna(subset=['PTDOBYY']).set_index('sid')
adni_ppmi['age_at_image_taken'] = pd.to_datetime(adni_ppmi.index.map(lambda x: x.split('-')[2])).map(lambda x: x.year).values - adni_ppmi['PTDOBYY']

In [9]:
dementia_subjects = raw_data[raw_data['study_diagnosis']=='ADNI_Dementia']['subject_id']
mci_subjects = raw_data[raw_data['study_diagnosis']=='ADNI_MCI']['subject_id']
intersection_subjects = list(set(dementia_subjects).intersection(set(mci_subjects)))
print (len(intersection_subjects))
print (len(set(mci_subjects)), len(set(dementia_subjects)))

335
1149 731


In [10]:
mci_and_dementia = adni_ppmi[adni_ppmi['subject_id'].isin(intersection_subjects)]
mci_only_subjects = set(mci_subjects).difference(intersection_subjects)
mci_only = adni_ppmi[adni_ppmi['subject_id'].isin(mci_only_subjects)]
mci_and_dementia['time_from_diagnosis'] = mci_and_dementia['age_ADDX'] - mci_and_dementia['age_at_image_taken']

In [11]:
req_columns = ['sex', 'presentage', 'IID', 'brainVolume', 'ens_ad_severity_scores', 'ens_pd_severity_scores', 'ens_logit_ad_severity_scores', 'ens_logit_pd_severity_scores']
L_mci_only = pd.merge(phenotypes[req_columns].copy(), mci_only, left_index=True, right_index=True)
L_mci_only.shape

(3411, 997)

In [12]:
# phenotypes

In [13]:
req_columns = ['sex', 'presentage', 'IID', 'brainVolume', 'ens_ad_severity_scores', 'ens_pd_severity_scores', 'ens_logit_ad_severity_scores', 'ens_logit_pd_severity_scores']
L_mci_and_dementia = pd.merge(phenotypes[req_columns].copy(), mci_and_dementia, left_index=True, right_index=True)
L_mci_and_dementia.shape

(1752, 998)

In [14]:
subject_times = L_mci_only.groupby('IID').agg('max')['age_at_image_taken'] - L_mci_only.groupby('IID').agg('min')['age_at_image_taken']
subject_times = dict(zip(list(subject_times.index), list(subject_times.values)))

In [15]:
L_mci_only['time_from_diagnosis'] = L_mci_only['IID'].map(subject_times)

In [16]:
L_mci_only['time_from_diagnosis'].isna().sum()

0

In [17]:
L = pd.concat([L_mci_only, L_mci_and_dementia], axis=0)
required_columns = ['time_from_diagnosis', 'subject_id', 'age_at_image_taken', 'sex', 'brainVolume', 'ens_ad_severity_scores', 'ens_pd_severity_scores', 'ens_logit_ad_severity_scores', 'ens_logit_pd_severity_scores']
L[required_columns].sort_values(by=['subject_id', 'age_at_image_taken']).drop_duplicates(subset=['subject_id'], keep='first')
S = L.sort_values(by=['subject_id', 'age_at_image_taken']).drop_duplicates(subset=['subject_id'], keep='first')
S['event_col'] = S['subject_id'].map(lambda x: 0 if x in mci_only_subjects else 1)


In [18]:
sel_invicro_features_filtered = list(adata.var.index)
sel_invicro_features_filtered = ['ens_logit_ad_severity_scores', 'ens_logit_pd_severity_scores']

In [19]:
from lifelines import CoxPHFitter
summary_list = []
for col in tqdm(sel_invicro_features_filtered):
    formula = f"age_at_image_taken + sex + brainVolume + {col}"
    T = S[~(S['time_from_diagnosis'] < 0)].copy()
    # T['event_col'] = T['subject_id'].map(lambda x: 1 if x>=0 else 0)
    T = T.dropna(subset=formula.split(' + ') + ['time_from_diagnosis', 'event_col'])
    # T[col] = (T[col] - T[col].mean()) / T[col].std()
    T = T[formula.split(' + ') + ['time_from_diagnosis', 'event_col']]
    # T['time_from_diagnosis'] = T['time_from_diagnosis'].fillna(10)
    cph = CoxPHFitter()
    cph.fit(T, duration_col='time_from_diagnosis', event_col='event_col', formula=formula)
    # cph.print_summary()
    summary_list.append(cph.summary.loc[[col]])
    break

  0%|                                                                                                                                                                                                                         | 0/2 [00:00<?, ?it/s]


In [20]:
cph.print_summary()

<lifelines.CoxPHFitter: fitted with 749 total observations, 708 right-censored observations>
             duration col = 'time_from_diagnosis'
                event col = 'event_col'
      baseline estimation = breslow
   number of observations = 749
number of events observed = 41
   partial log-likelihood = -213.42
         time fit was run = 2023-05-10 17:14:18 UTC

---
                               coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                            
age_at_image_taken            -0.04       0.96       0.02            -0.09             0.00                 0.92                 1.00
brainVolume                   -1.13       0.32       3.02            -7.04             4.78                 0.00               118.80
ens_logit_ad_severity_scores   0.41       1.50       0.05             0.31             0.50                 1.37                 1.65
sex                            0.06       1.06       0.40            -0.73             0.84                 0.48                 2.33

                               cmp to     z      p   -log2(p)
covariate                                                    
age_at_image_taken               0.00 -1.81   0.07       3.83
brainVolume                      0.00 -0.38   0.71       0.50
ens_logit_ad_severity_scores     0.00  8.37 <0.005      53.95
sex                              0.00  0.15   0.88       0.18
---
Concordance = 0.86
Partial AIC = 434.84
log-likelihood ratio test = 67.54 on 4 df
-log2(p) of ll-ratio test = 43.60

In [21]:
pd.concat(summary_list).sort_values(by='p').head(n=60)

,coef,exp(coef),se(coef),coef lower 95%,coef upper 95%,exp(coef) lower 95%,exp(coef) upper 95%,cmp to,z,p,-log2(p)
covariate,,,,,,,,,,,
ens_logit_ad_severity_scores,0.407011,1.50232,0.048627,0.311704,0.502317,1.36575,1.652546,0.0,8.370086,5.757600e-17,53.94731


In [22]:
T.to_csv("data/MCI_survival.csv", index=False)

In [23]:
formula

'age_at_image_taken + sex + brainVolume + ens_logit_ad_severity_scores'